In [12]:
import pandas as pd
import glob
import os

# ====================================================
# [설정] 통합할 CSV 파일들이 있는 폴더
# ====================================================
# 역슬래시(\) 문제 방지를 위해 앞에 r을 붙여주었습니다.
target_folder = r"C:\ai\lecNote\10_1stTeamProject\naver_news\648_비즈워치"

# ====================================================
# 1. 파일 목록 확인
# ====================================================
file_pattern = os.path.join(target_folder, "*.csv")
all_files = glob.glob(file_pattern)

# 기존에 합쳐진 결과 파일이 있다면 병합 대상에서 제외 (중복 방지)
# (이번엔 결과 파일명이 동적으로 바뀌지만, 혹시 모를 merged_result.csv 등도 제외)
all_files = [f for f in all_files if "merged_" not in f and ".pkl" not in f]

print(f"📂 폴더 위치: {target_folder}")
print(f"🔎 발견된 CSV 파일: {len(all_files)}개")

if len(all_files) == 0:
    print("❌ 파일을 하나도 못 찾았습니다! 경로를 다시 확인해 주세요.")
    exit()

# ====================================================
# 2. 파일 순회, 정보 추출 및 병합
# ====================================================
df_list = []
collected_oids = set() # 언론사 코드(OID)를 모을 집합 (중복 제거)
collected_dates = []   # 날짜들을 모을 리스트

print("\n🚀 병합 작업을 시작합니다...")

for i, filename in enumerate(all_files):
    try:
        base_name = os.path.basename(filename)
        
        # --- [추가됨] 파일명에서 OID와 날짜 추출 로직 ---
        # 예: oid=018&date=20251220.csv
        try:
            # 확장자 제거 후 '&'로 분리 -> ['oid=018', 'date=20251220']
            name_parts = os.path.splitext(base_name)[0].split('&')
            
            for part in name_parts:
                if "oid=" in part:
                    collected_oids.add(part.replace("oid=", ""))
                elif "date=" in part:
                    collected_dates.append(part.replace("date=", ""))
        except:
            pass # 파일명 형식이 다르면 정보 추출만 건너뜀
        # ----------------------------------------------

        # 1) 파일 읽기
        try:
            temp_df = pd.read_csv(filename, encoding='utf-8-sig')
        except UnicodeDecodeError:
            temp_df = pd.read_csv(filename, encoding='cp949')
        
        # 2) 데이터가 비어있는지 확인
        if temp_df.empty:
            print(f"   ⚠️ [Skip] 빈 파일입니다: {base_name}")
            continue

        # 3) 리스트에 추가
        df_list.append(temp_df)
        
        print(f"   [{i+1}/{len(all_files)}] 읽기 성공: {base_name} (데이터 {len(temp_df)}건)")

    except Exception as e:
        print(f"   ❌ 읽기 에러: {os.path.basename(filename)} -> {e}")

# ====================================================
# 3. 데이터프레임 합치기 & 날짜 변환
# ====================================================
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"\n📊 [1차 완료] 총 {len(combined_df)}개의 데이터를 합쳤습니다.")
    
    # 날짜 컬럼 타입 변환 (Object -> Datetime)
    if '날짜' in combined_df.columns:
        print("🛠️ 날짜 포맷 변환 중... (Text -> Datetime)")
        combined_df['날짜'] = pd.to_datetime(combined_df['날짜'], errors='coerce')
        
        # 날짜순 정렬
        combined_df = combined_df.sort_values(by='날짜').reset_index(drop=True)
        print(f"   -> 변환 완료! 데이터 타입: {combined_df['날짜'].dtype}")
    else:
        print("⚠️ '날짜' 컬럼이 없어 변환을 건너뜁니다.")

    # ====================================================
    # 4. 동적 파일명 생성 및 Pickle 저장
    # ====================================================
    # OID 결정 (여러 개면 첫 번째 것 사용, 없으면 unknown)
    final_oid = list(collected_oids)[0] if collected_oids else "unknown"
    
    # 날짜 범위 결정
    if collected_dates:
        collected_dates.sort() # 날짜 정렬
        start_date = collected_dates[0]
        end_date = collected_dates[-1]
        date_range_str = f"{start_date}-{end_date}"
    else:
        date_range_str = "no_dates"
    
    # 최종 파일명 생성: 018_20251201-20251221.pkl
    output_name = f"{final_oid}_{date_range_str}.pkl"
    save_path = os.path.join(target_folder, output_name)
    
    # Pickle로 저장
    combined_df.to_pickle(save_path)
    
    print(f"\n✅ 모든 작업 끝! 저장된 파일: {save_path}")
    print(f"   (파일명 정보: 언론사 {final_oid}, 기간 {date_range_str})")
    print(combined_df.head()) 

else:
    print("\n❌ 합칠 데이터가 없습니다.")

📂 폴더 위치: C:\ai\lecNote\10_1stTeamProject\naver_news\648_비즈워치
🔎 발견된 CSV 파일: 21개

🚀 병합 작업을 시작합니다...
   [1/21] 읽기 성공: oid=648&date=20251201.csv (데이터 46건)
   [2/21] 읽기 성공: oid=648&date=20251202.csv (데이터 38건)
   [3/21] 읽기 성공: oid=648&date=20251203.csv (데이터 40건)
   [4/21] 읽기 성공: oid=648&date=20251204.csv (데이터 45건)
   [5/21] 읽기 성공: oid=648&date=20251205.csv (데이터 30건)
   [6/21] 읽기 성공: oid=648&date=20251206.csv (데이터 20건)
   [7/21] 읽기 성공: oid=648&date=20251207.csv (데이터 20건)
   [8/21] 읽기 성공: oid=648&date=20251208.csv (데이터 37건)
   [9/21] 읽기 성공: oid=648&date=20251209.csv (데이터 26건)
   [10/21] 읽기 성공: oid=648&date=20251210.csv (데이터 40건)
   [11/21] 읽기 성공: oid=648&date=20251211.csv (데이터 26건)
   [12/21] 읽기 성공: oid=648&date=20251212.csv (데이터 27건)
   [13/21] 읽기 성공: oid=648&date=20251213.csv (데이터 20건)
   [14/21] 읽기 성공: oid=648&date=20251214.csv (데이터 20건)
   [15/21] 읽기 성공: oid=648&date=20251215.csv (데이터 38건)
   [16/21] 읽기 성공: oid=648&date=20251216.csv (데이터 38건)
   [17/21] 읽기 성공: oid=648&date=20251217.csv (데이